In [1]:
import pandas as pd
import numpy as np
import gzip
import pickle
from os.path import expanduser
from copy import deepcopy
from sampacking.partition import manage_lists, reshuffle_bins, bin_packing_mean, check_packing, ks_2samp_multi_dim, add_delta_equal_weights
%load_ext autoreload
%autoreload 1
%aimport sampacking.partition

In [2]:
od = pickle.load(open('../data/partition_data_99.p', 'rb'))

In [3]:
ordered_data = od
ordered_weights = [x.shape[0] for x in od]
print(len(od))
number_of_bins = 6

459


In [4]:
inds = bin_packing_mean(ordered_data, number_of_bins, distance_func=ks_2samp_multi_dim)
print(check_packing(inds, ordered_weights, ordered_data, ks_2samp_multi_dim))
ww = [[ordered_weights[i] for i in ii] for ii in inds]
print(list(map(lambda x: sum(x)*len(x), ww)))

(206233.83333333334, 2580.6494929679147, 0.0045179796658785243, 0.0080547618810614674)
[207288, 205725, 208901, 201318, 208797, 205374]


In [5]:
iis = reshuffle_bins(inds, ordered_data, 0.5, ks_2samp_multi_dim)
print(len(iis))

(206233.83333333334, 2580.6494929679147, 0.0045179796658785243, 0.0080547618810614674) ; std/mean = 0.012513220800181904
(206228.5, 1585.3458876009781, 0.004697706090530121, 0.0070823086471373253) ; std/mean = 0.007687326861229064
(206244.0, 236.90504426879559, 0.0042381932844912651, 0.0066011552643445714) ; std/mean = 0.0011486639333449487
6


In [6]:
ww = [[ordered_weights[i] for i in ii] for ii in iis]
print(list(map(lambda x: sum(x)*len(x), ww)))

[206496, 205800, 206514, 206190, 206232, 206232]


In [7]:
ww = add_delta_equal_weights(ordered_weights)
ww[:5], ordered_weights[:5]

[13, 33, 26, 139, 11] [280  80 142 316 448]
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
1
99
98


([13.001099999999999, 33.0002, 26.000299999999999, 139.0, 11.0008],
 [13, 33, 26, 139, 11])

In [8]:
def bin_packing_mod(pdfs_input, number_bins, distance_func=ks_2samp_multi_dim):
    """
    partition list of numpy arrays pdfs_input into a list of lists

    :param pdfs_input:
    :param number_bins: tentative number of bins
    :param distance_func: sample distance func
    :return:
    """

    # descending order in size and in std
    inds = [x[0] for x in sorted(enumerate(pdfs_input),
                                 key=lambda y: (y[1].shape[0], y[1].std()), reverse=True)]

    weights = [pdfs_input[ii].shape[0] for ii in inds]
    pdfs = [pdfs_input[ii] for ii in inds]

    w_mean0 = mean(weights)
    sample0 = concatenate(pdfs_input)
    items_per_bin = int(ceil(len(weights) / number_bins))
    bin_capacity = (items_per_bin * w_mean0)
    bin_product = bin_capacity * items_per_bin

    if max(weights) > bin_capacity:
        raise ValueError('Max item weight is greater than proposed bin cap')
    # populate each bin with a largest available element
    bins = [[x] for x in weights[:number_bins]]
    indices = [[i] for i in inds[:number_bins]]
    pdf_bins = [[i] for i in pdfs[:number_bins]]

    indices_output = []
    weights = weights[number_bins:]
    inds = inds[number_bins:]
    pdfs = pdfs[number_bins:]

    diffs = [x - y for x, y in zip(weights[:-1], weights[1:])]
    bbs = [0] + [j + 1 for j in range(len(diffs)) if diffs[j] != 0]
    # pdfs2 = [pdfs[bbs[i]:bbs[i + 1]] for i in range(len(bbs) - 1)] + [pdfs[bbs[-1]:]]
    # inds2 = [inds[bbs[i]:bbs[i + 1]] for i in range(len(bbs) - 1)] + [inds[bbs[-1]:]]
    weights_uni = [weights[bbs[i]] for i in range(len(bbs))]
    j_cur_bin = 0
    k_cur_weight = 0

    state = -1, -1, -1, -1
    while weights_uni:
        ind_bin = indices[j_cur_bin]
        wei_bin = bins[j_cur_bin]
        pdf_bin = pdf_bins[j_cur_bin]
        ind_strata = inds2[k_cur_weight]
        wei_strata = weights_uni[k_cur_weight]
        pdf_strata = pdfs2[k_cur_weight]

        pi_tentative = (sum(wei_bin) + wei_strata) * (len(wei_bin) + 1)
        pi_tentative_min = (sum(wei_bin) + min(weights_uni)) * (len(wei_bin) + 1)
        if pi_tentative < bin_product:
            dists = []
            for pdf in pdf_strata:
                bin_pdf_dist = distance_func(concatenate(pdf_bin), sample0)[0]
                bin_pdf_dist_new = distance_func(concatenate(pdf_bin + [pdf]), sample0)[0]
                diff_pdf = bin_pdf_dist_new / bin_pdf_dist
                dists.append(diff_pdf)
            j_best = argmin(array(dists) ** 2)
            ind_bin.append(ind_strata.pop(j_best))
            pdf_bin.append(pdf_strata.pop(j_best))
            wei_bin.append(wei_strata)
            accepted = True
            state = j_cur_bin, k_cur_weight, len(bins), len(weights_uni)
        else:
            accepted = False
        if not accepted and state == (j_cur_bin, k_cur_weight, len(bins), len(weights_uni)):
            print('Loop detected')
            indices.append([ind_strata.pop()])
            pdf_bins.append([pdf_strata.pop()])
            bins.append([wei_strata])
        if not ind_strata:
            inds2.pop(k_cur_weight)
            pdfs2.pop(k_cur_weight)
            weights_uni.pop(k_cur_weight)
        if pi_tentative_min > bin_product:
            indices_output.append(indices.pop(j_cur_bin))
            pdf_bins.pop(j_cur_bin)
            bins.pop(j_cur_bin)
        if weights_uni:
            k_cur_weight = (k_cur_weight + 1) % len(weights_uni)
        if bins:
            j_cur_bin = (j_cur_bin + 1) % len(bins)
    indices_output.extend(indices)
    return indices_output

NameError: name 'ks_2samp' is not defined